In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt

In [3]:
# Path to the CSV file
file_path = '/home/ubuntu/MoCap/T1_1Act_quarterComp/T1_1act_quarterComp.csv'

# Read the file and find the start of frame data
with open(file_path, 'r') as file:
    lines = file.readlines()

frame_start_index = None
for index, line in enumerate(lines):
    if line.startswith('frame'):
        frame_start_index = index
        break



In [7]:
all_positions = []  # This will store a list of positions for each frame
time_stamps = []

# Assuming `lines` and `frame_start_index` have been defined as per your previous snippets.
for line in lines[frame_start_index:]:
    parts = line.split(',')
    if parts[0] != 'frame':
        continue
    
    frame_time = float(parts[2])
    time_stamps.append(frame_time)
    
    num_points = int(parts[4])
    frame_positions = []  # Store positions for this frame only
    for i in range(num_points):
        base_index = 5 + i * 5
        x = float(parts[base_index])
        y = float(parts[base_index + 1])
        z = float(parts[base_index + 2])
        index = int(parts[base_index + 3])
        frame_positions.append((x, y, z, index))
    
    all_positions.append(frame_positions)  # Append list of tuples for each frame

In [8]:

def update_plot(frame_index):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Convert current frame positions to a NumPy array
    positions = np.array(all_positions[frame_index])
    
    ax.scatter(
        positions[:, 0],
        positions[:, 1],
        positions[:, 2],
        cmap='viridis'
    )

    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_zlabel('Z Coordinate')
    ax.set_title(f'Interactive Plot of Marker Positions at Frame {frame_index}')
    plt.show()

# Slider to select the frame to display
frame_slider = widgets.IntSlider(
    min=0, 
    max=len(all_positions) - 1, 
    step=1, 
    value=0, 
    description='Select Frame:'
)

# Interactive widget
out = widgets.interactive_output(update_plot, {'frame_index': frame_slider})

# Display the interactive plot with slider
display(widgets.VBox([frame_slider, out]))

In [12]:
# Define a list of colors
colors = [
    "aliceblue", "antiquewhite", "aqua", "aquamarine", "azure",
    "beige", "bisque", "black", "blanchedalmond", "blue",
    "blueviolet", "brown", "burlywood", "cadetblue",
    "chartreuse", "chocolate", "coral", "cornflowerblue",
    "cornsilk", "crimson", "cyan", "darkblue", "darkcyan",
    "darkgoldenrod", "darkgray", "darkgrey", "darkgreen",
    "darkkhaki", "darkmagenta", "darkolivegreen", "darkorange",
    "darkorchid", "darkred", "darksalmon", "darkseagreen",
    "darkslateblue", "darkslategray", "darkslategrey",
    "darkturquoise", "darkviolet", "deeppink", "deepskyblue",
    "dimgray", "dimgrey", "dodgerblue", "firebrick",
    "floralwhite", "forestgreen", "fuchsia", "gainsboro",
    "ghostwhite", "gold", "goldenrod", "gray", "grey", "green",
    "greenyellow", "honeydew", "hotpink", "indianred", "indigo",
    "ivory", "khaki", "lavender", "lavenderblush", "lawngreen",
    "lemonchiffon", "lightblue", "lightcoral", "lightcyan",
    "lightgoldenrodyellow", "lightgray", "lightgrey",
    "lightgreen", "lightpink", "lightsalmon", "lightseagreen",
    "lightskyblue", "lightslategray", "lightslategrey",
    "lightsteelblue", "lightyellow", "lime", "limegreen",
    "linen", "magenta", "maroon", "mediumaquamarine",
    "mediumblue", "mediumorchid", "mediumpurple",
    "mediumseagreen", "mediumslateblue", "mediumspringgreen",
    "mediumturquoise", "mediumvioletred", "midnightblue",
    "mintcream", "mistyrose", "moccasin", "navajowhite", "navy",
    "oldlace", "olive", "olivedrab", "orange", "orangered",
    "orchid", "palegoldenrod", "palegreen", "paleturquoise",
    "palevioletred", "papayawhip", "peachpuff", "peru", "pink",
    "plum", "powderblue", "purple", "red", "rosybrown",
    "royalblue", "rebeccapurple", "saddlebrown", "salmon",
    "sandybrown", "seagreen", "seashell", "sienna", "silver",
    "skyblue", "slateblue", "slategray", "slategrey", "snow",
    "springgreen", "steelblue", "tan", "teal", "thistle", "tomato",
    "turquoise", "violet", "wheat", "white", "whitesmoke",
    "yellow", "yellowgreen"
]

# Extract marker indices from your data
marker_indices = [point[3] for frame in all_positions for point in frame]
unique_indices = list(set(marker_indices))

# Create a color mapping for each unique marker index, cycling through the color list
index_to_color = {index: colors[i % len(colors)] for i, index in enumerate(unique_indices)}

# Map each marker's index to a color
marker_colors = [index_to_color[idx] for idx in marker_indices]

# Extract the (x, y, z) coordinates
all_points = [point[:3] for frame in all_positions for point in frame]  # Ignore the marker index this time
all_points = np.array(all_points)

# Check if there are points to plot
if all_points.size == 0:
    print("No points to plot.")
else:
    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=all_points[:, 0],
        y=all_points[:, 1],
        z=all_points[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=marker_colors,  # Use the manually defined colors
            opacity=0.8
        )
    )])

    # Update plot layout
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title='X Coordinate',
            yaxis_title='Y Coordinate',
            zaxis_title='Z Coordinate'
        ),
        title='All Marker Positions Across All Frames with Distinct Colors'
    )

    # Show the plot
    fig.show()

ValueError: 
    Invalid element(s) received for the 'color' property of scatter3d.marker
        Invalid elements include: ['peach', 'peach', 'peach', 'peach', 'peach', 'peach', 'peach', 'peach', 'peach', 'peach']

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslateg

In [41]:
# Initialize containers for the filtered points and outliers
filtered_positions = []
outlier_positions = []
filtered_time_stamps = []

# Process each frame for outlier detection
for frame, time_stamp in zip(all_positions, time_stamps):
    if len(frame) > 0:
        points = np.array(frame)  # Convert frame to a numpy array
        
        # Calculate Q1, Q3, and IQR
        Q1 = np.percentile(points, 25, axis=0)
        Q3 = np.percentile(points, 75, axis=0)
        IQR = Q3 - Q1
        
        # Define lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Determine inliers and outliers
        is_inlier = (points >= lower_bound) & (points <= upper_bound)
        inliers = points[np.all(is_inlier, axis=1)]
        outliers = points[~np.all(is_inlier, axis=1)]

        # Append the results to their respective lists
        if inliers.size > 0:
            filtered_positions.append(inliers.tolist())
            filtered_time_stamps.append(time_stamp)
        if outliers.size > 0:
            outlier_positions.append(outliers.tolist())


In [46]:
# Assuming 'filtered_positions' and 'outlier_positions' contain lists of 3D coordinates for each frame
# Flatten the lists of lists into single lists for Plotly plotting
filtered_points = np.vstack(filtered_positions)  # Stacking all filtered points
outlier_points = np.vstack(outlier_positions)  # Stacking all outlier points

# Create a Plotly graph object for the interactive plot
fig = go.Figure()

# Add the filtered points in blue
fig.add_trace(go.Scatter3d(
    x=filtered_points[:, 0],
    y=filtered_points[:, 1],
    z=filtered_points[:, 2],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',  # Use blue to indicate filtered points
        opacity=0.8
    ),
    name='Filtered Points'
))

# Add the outliers in red
fig.add_trace(go.Scatter3d(
    x=outlier_points[:, 0],
    y=outlier_points[:, 1],
    z=outlier_points[:, 2],
    mode='markers',
    marker=dict(
        size=3,
        color='red',  # Use red to indicate outliers
        opacity=0.8
    ),
    name='Outliers'
))

# Update the layout to create a visually appealing and informative plot
fig.update_layout(
    title='Interactive 3D Scatter Plot of MoCap Data',
    scene=dict(
        xaxis_title='X Coordinates',
        yaxis_title='Y Coordinates',
        zaxis_title='Z Coordinates'
    ),
    legend_title_text='Point Type'
)

# Show the plot

plot4_path = "scrollabledistanceplot.html"
fig.write_html(plot4_path)

In [43]:

filtered_avg_positions = []
# Assume 'all_positions' is a list of numpy arrays, where each array contains points for a frame
for frame in all_positions:
    if len(frame) > 0:  # Ensure there are points in the frame
        points = np.array(frame)
        average_position = np.mean(points, axis=0)  # Compute the mean along each column (x, y, z)
        filtered_avg_positions.append(average_position.tolist())
    else:
        filtered_avg_positions.append([np.nan, np.nan, np.nan])  # Append NaNs if frame is empty


In [44]:
# Define a function to apply a Butterworth low-pass filter
def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs  # Nyquist Frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    return y

# Sample frequency and cutoff frequency
fs = 30  # Sample rate, Hz (adjust based on your data sampling rate)
cutoff = 3  # Desired cutoff frequency of the filter, Hz

# Apply the filter
filtered_avg_positions = butter_lowpass_filter(filtered_avg_positions, cutoff, fs, order=5)

In [45]:


# Convert list of average positions to a NumPy array for easier handling
filtered_avg_positions = np.array(filtered_avg_positions)

# Create a Plotly figure
fig = go.Figure(data=[go.Scatter3d(
    x=filtered_avg_positions[:, 0],
    y=filtered_avg_positions[:, 1],
    z=filtered_avg_positions[:, 2],
    mode='lines+markers',
    marker=dict(
        size=5,
        color=np.linspace(0, filtered_time_stamps[-1], len(filtered_time_stamps)),  # Gradient color based on time
        colorscale='Viridis',
        showscale=True
    ),
    line=dict(
        color='#1f77b4',  # Consistent color for the line
        width=2
    )
)])

# Update plot layout for better visualization
fig.update_layout(
    title='Trajectory of Average Positions Over Time',
    scene=dict(
        xaxis_title='X Coordinates',
        yaxis_title='Y Coordinates',
        zaxis_title='Z Coordinates'
    ),
    legend_title_text='Frame'
)

# Show the interactive plot
fig.show()